# Job Hangs/Stuck Troubleshooting**Symptoms:** Spark UI показывает Running но ничего не происходит

In [ ]:
from pyspark.sql import SparkSessionspark = SparkSession.builder.remote("sc://spark-connect:15002").getOrCreate()

## Диагностика```python# Проверить active taskssc = spark.sparkContextstatus = sc.statusTracker()print(f"Active jobs: {status.getActiveJobsIds()}")```## Возможные причины:1. Network policies блокируют executor-driver2. Skew partition (одна задача обрабатывается слишком долго)3. Deadlock## Решение```bash# Проверить network policieskubectl get networkpolicies -n spark# Добавить policy если нужноkubectl apply -f -<<EOFapiVersion: networking.k8s.io/v1kind: NetworkPolicymetadata:  name: spark-executorsspec:  podSelector:    matchLabels:      spark-role: executor  policyTypes:  - Egress  egress:  - to:    - podSelector:        matchLabels:          spark-role: driver    ports:    - protocol: TCP      port: 7078EOF```

In [ ]:
spark.stop()